In [1]:
!pip install timm
!pip install transformers
!pip install sentence-transformers
!pip install numba
!pip install tensorboard
!pip install torchmetrics
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/CLIP Project
%matplotlib inline
%load_ext tensorboard
!unzip "Train/resized_train.zip" -d ""
#!tensorboard --logdir=runs --bind_all # http://localhost:6006/

Streaming output truncated to the last 5000 lines.
  inflating: resized_train/ImageCLEFmedCaption_2022_train_078714.jpg  
  inflating: resized_train/ImageCLEFmedCaption_2022_train_078715.jpg  
  inflating: resized_train/ImageCLEFmedCaption_2022_train_078716.jpg  
  inflating: resized_train/ImageCLEFmedCaption_2022_train_078717.jpg  
  inflating: resized_train/ImageCLEFmedCaption_2022_train_078718.jpg  
  inflating: resized_train/ImageCLEFmedCaption_2022_train_078719.jpg  
  inflating: resized_train/ImageCLEFmedCaption_2022_train_078720.jpg  
  inflating: resized_train/ImageCLEFmedCaption_2022_train_078721.jpg  
  inflating: resized_train/ImageCLEFmedCaption_2022_train_078722.jpg  
  inflating: resized_train/ImageCLEFmedCaption_2022_train_078723.jpg  
  inflating: resized_train/ImageCLEFmedCaption_2022_train_078724.jpg  
  inflating: resized_train/ImageCLEFmedCaption_2022_train_078725.jpg  
  inflating: resized_train/ImageCLEFmedCaption_2022_train_078726.jpg  
  inflating: resized_train

In [2]:
import os
import cv2
import gc
import numpy as np
import pandas as pd
import itertools
from tqdm.autonotebook import tqdm
import albumentations as A
import matplotlib.pyplot as plt
from operator import itemgetter

import torch
from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter
from torch import nn
import torch.nn.functional as F
import timm
from transformers import DistilBertModel, DistilBertConfig, DistilBertTokenizer
from sentence_transformers import SentenceTransformer, util
from torchmetrics import ConfusionMatrix
# from numba import cuda 
# device = cuda.get_current_device()
# device.reset()
# %tensorboard --logdir runs

<ipython-input-2-3e1da2bf1718>:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


## Config

In [3]:
for model in timm.list_models('*convnext*'):
  print(model)

convnext_atto
convnext_atto_ols
convnext_base
convnext_base_384_in22ft1k
convnext_base_in22ft1k
convnext_base_in22k
convnext_femto
convnext_femto_ols
convnext_large
convnext_large_384_in22ft1k
convnext_large_in22ft1k
convnext_large_in22k
convnext_nano
convnext_nano_ols
convnext_pico
convnext_pico_ols
convnext_small
convnext_small_384_in22ft1k
convnext_small_in22ft1k
convnext_small_in22k
convnext_tiny
convnext_tiny_384_in22ft1k
convnext_tiny_hnf
convnext_tiny_in22ft1k
convnext_tiny_in22k
convnext_xlarge_384_in22ft1k
convnext_xlarge_in22ft1k
convnext_xlarge_in22k


In [4]:
class CFG:
    debug = False
    image_path = 'resized_train' #'Train/resized_train'
    captions_path = 'Train/caption_prediction_train.csv'
    batch_size = 64
    validation_ratio = 0.1
    num_workers = 2
    head_lr = 1e-3
    image_encoder_lr = 1e-4
    image_concept_encoder_lr = 1e-4
    text_encoder_lr = 1e-5
    weight_decay = 1e-3
    patience = 1
    factor = 0.8
    epochs = 50
    pre_train_epochs = 10
    num_of_concepts = 200
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # writer = SummaryWriter(comment="convnext_nano")
    writer_comment = 'transfer_learning_removed_sigmoid'

    model_name = 'convnext_nano'
    num_classes = num_of_concepts # 8374, Number of concepts
    image_embedding = timm.create_model(model_name, pretrained=True).forward_features(torch.randn(2, 3, 299, 299)).shape[1] if  num_classes == 0 else num_classes # 299, 299
    text_encoder_model = 'distilbert-base-uncased'
    text_embedding = 768
    text_tokenizer = 'distilbert-base-uncased'
    max_length = 200
    samples = 10000

    pretrained = True # for both image encoder and text encoder
    trainable = True # for both image encoder and text encoder
    temperature = 1.0

    # image size
    size = 224

    # for projection head; used for both image and text encoders
    num_projection_layers = 1
    projection_dim = 256 
    dropout = 0.1

    similarity_threshold = 0.8


## Utils

In [5]:
class MultiClassAvgMeter:
    def __init__(self, name="Metric"):
        self.name = name
        self.reset()

    def reset(self):
        self.avg_loss, self.loss_sum, self.count = [0] * 3
        self.TN, self.FP, self.FN, self.TP = [0] * 4
        self.accuracy, self.precision, self.recall, self.F1 = [0] * 4

    def update(self, val, count, TN, FP, FN, TP):
        self.count += count
        self.loss_sum += val * count
        self.avg_loss = self.loss_sum / self.count

        self.TN += TN
        self.FP += FP
        self.FN += FN
        self.TP += TP

        self.accuracy = (self.TN + self.TP) / (self.TN + self.FP + self.FN + self.TP)
        self.precision = self.TP / (self.FP + self.TP)
        self.recall = self.TP / (self.TP + self.FN)
        self.F1 = (2 * self.precision * self.recall) / (self.precision + self.recall)

    def __repr__(self):
        text = f"{self.name}: {self.avg_loss:.4f}, {self.accuracy:.4f}, {self.precision:.4f},  {self.recall:.4f},  {self.F1:.4f}"
        return text
  
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group["lr"]


In [6]:
df_train_concepts = pd.read_csv('Train/concept_detection_train.csv', sep='\t')
df_concepts = pd.read_csv('Train/concepts.csv', sep='\t')

concept_count = {}
max_num = 0

for train_id, caption_train_row in df_train_concepts.iterrows():
  cuis = caption_train_row['cuis'].split(';')
  max_num = max(max_num, len(cuis))
  for cui in cuis:
    concept_name = cui
    if concept_name in concept_count:
      concept_count[concept_name] += 1
    else:
      concept_count[concept_name] = 1

top_concepts = dict(sorted(concept_count.items(), key=lambda x:x[1], reverse = True)[:CFG.num_of_concepts]).keys()

def get_concept_2_index():
  concept_2_index = {} # {'C2825493': 0 ... 'C0231462': 8373}
  counter = 0
  for i, (concept_id, concept_row) in enumerate(df_concepts.iterrows()):
    if concept_row['concept'] in top_concepts:
      concept_2_index[concept_row['concept']] = counter
      counter += 1
  return concept_2_index

def get_concept_one_hot_encoding(img_id, concept_2_index):
  one_hot_encoding = torch.zeros(len(concept_2_index))
  cuis = df_train_concepts.loc[df_train_concepts['ID'] == img_id, 'cuis'].values[0].split(';')
  for cui in cuis:
    if cui in top_concepts:
      one_hot_encoding[concept_2_index[cui]] = 1
  return one_hot_encoding

## Dataset

In [7]:
class CLIPDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe, tokenizer, transforms):
        """
        image_filenames and cpations must have the same length; so, if there are
        multiple captions for each image, the image_filenames must have repetitive
        file names 
        """

        self.image_filenames = dataframe['ID'].values
        self.captions = dataframe['caption'].values
        self.encoded_captions = tokenizer(
            list(self.captions), padding=True, truncation=True, max_length=CFG.max_length
        )
        self.transforms = transforms
        self.concept_2_index = get_concept_2_index()

    def __getitem__(self, idx):
        item = {
            key: torch.tensor(values[idx])
            for key, values in self.encoded_captions.items()
        }

        image = cv2.imread(f"{CFG.image_path}/{self.image_filenames[idx] + '.jpg'}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = self.transforms(image=image)['image']
        item['image'] = torch.tensor(image).permute(2, 0, 1).float()
        item['caption'] = self.captions[idx]
        item['concept'] = get_concept_one_hot_encoding(self.image_filenames[idx], self.concept_2_index)
        return item


    def __len__(self):
        return len(self.captions)



def get_transforms(mode="train"):
    if mode == "train":
        return A.Compose(
            [
                A.Resize(CFG.size, CFG.size, always_apply=True),
                A.Normalize(max_pixel_value=255.0, always_apply=True),
            ]
        )
    else:
        return A.Compose(
            [
                A.Resize(CFG.size, CFG.size, always_apply=True),
                A.Normalize(max_pixel_value=255.0, always_apply=True),
            ]
        )

## Image Encoder

In [8]:
class ImageEncoder(nn.Module):
    """
    Encode images to a fixed size vector
    """

    def __init__(
        self, model_name=CFG.model_name, pretrained=CFG.pretrained, trainable=CFG.trainable
    ):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained, num_classes=CFG.num_classes, global_pool="avg")
        for p in self.model.parameters():
            p.requires_grad = trainable

    def forward(self, x):
        return self.model(x)

## Dataloader

In [9]:
def make_train_valid_dfs(add_concepts=False):
    dataframe = pd.read_csv('Train/caption_prediction_train.csv', sep='\t')
    # dataframe = dataframe.head(CFG.samples)
    # dataframe = pre_process_data(dataframe)
    # dataframe = add_concept_to_captions(dataframe) if add_concepts else dataframe
    
    max_id = dataframe.shape[0]
    image_ids = np.arange(0, max_id)
    np.random.seed(42)
    valid_ids = np.random.choice(
        image_ids, size=int(CFG.validation_ratio * len(image_ids)), replace=False
    )
    train_ids = [id_ for id_ in image_ids if id_ not in valid_ids]
    train_dataframe = dataframe.iloc[train_ids,:].reset_index()
    valid_dataframe = dataframe.iloc[valid_ids,:].reset_index()
    return train_dataframe, valid_dataframe


def build_loaders(dataframe, tokenizer, mode):
    transforms = get_transforms(mode=mode)
    dataset = CLIPDataset(
        dataframe=dataframe,
        tokenizer=tokenizer,
        transforms=transforms,
    )
    dataloader = torch.utils.data.DataLoader(
        dataset,
        batch_size=CFG.batch_size,
        num_workers=CFG.num_workers,
        shuffle=True if mode == "train" else False,
    )
    return dataloader

##Concept Pre-training

In [10]:
def criterion(loss_func, outputs, concept):
    losses = 0
    for i, key in enumerate(outputs):
      losses += loss_func(outputs[i], concept[i].to(CFG.device))
    return losses

def get_metrics(preds, target):
    confmat = ConfusionMatrix(task="binary", num_classes=2).to(CFG.device)
    confusion_matrix = confmat(preds, target.to(CFG.device))
    TN, FP, FN, TP = confusion_matrix[0][0], confusion_matrix[0][1], confusion_matrix[1][0], confusion_matrix[1][1] 
    return TN, FP, FN, TP

def pre_train_epoch(model, train_loader, optimizer, lr_scheduler, step, epoch):
    metrics = MultiClassAvgMeter()

    tqdm_object = tqdm(train_loader, total=len(train_loader))
    for i, batch in enumerate(tqdm_object):
        cuda_batch = {k: v.to(CFG.device) for k, v in batch.items() if k != 'caption'}
        cuda_batch['caption'] = batch['caption']
        outputs = model(cuda_batch['image'])

        loss = criterion(F.binary_cross_entropy, torch.sigmoid(outputs), batch['concept'])
        TN, FP, FN, TP = get_metrics(torch.round(torch.sigmoid(outputs)), batch['concept'])

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if step == 'batch':
            lr_scheduler.step()
        count = batch['image'].size(0)
        metrics.update(loss.item(), count, TN, FP, FN, TP)

        tqdm_object.set_postfix(train_loss = metrics.avg_loss, accuracy=metrics.accuracy, precision=metrics.precision, recall=metrics.recall, F1=metrics.F1, lr=get_lr(optimizer))
    return metrics


def pre_valid_epoch(model, valid_loader, epoch):
    metrics = MultiClassAvgMeter()

    tqdm_object = tqdm(valid_loader, total=len(valid_loader))
    for i, batch in enumerate(tqdm_object):
        cuda_batch = {k: v.to(CFG.device) for k, v in batch.items() if k != 'caption'}
        cuda_batch['caption'] = batch['caption']
        outputs = model(cuda_batch['image'])

        loss = criterion(F.binary_cross_entropy, torch.sigmoid(outputs), batch['concept'])
        TN, FP, FN, TP = get_metrics(torch.round(torch.sigmoid(outputs)), batch['concept'])

        count = batch['image'].size(0)
        metrics.update(loss.item(), count, TN, FP, FN, TP)

        tqdm_object.set_postfix(train_loss = metrics.avg_loss, accuracy=metrics.accuracy, precision=metrics.precision, recall=metrics.recall, F1=metrics.F1)
    return metrics


## Main

In [11]:
def main():
    train_df, valid_df = make_train_valid_dfs(add_concepts=True)
    tokenizer = DistilBertTokenizer.from_pretrained(CFG.text_tokenizer)
    train_loader = build_loaders(train_df, tokenizer, mode="train")
    valid_loader = build_loaders(valid_df, tokenizer, mode="valid")

    image_encoder = ImageEncoder().to(CFG.device)
    params = [{"params": image_encoder.parameters(), "lr": CFG.image_concept_encoder_lr}]
    optimizer = torch.optim.AdamW(params, weight_decay=0.)
    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", patience=CFG.patience, factor=CFG.factor)
    step = "epoch"

    best_F1 = float('-inf')
    consecutive = 0
    early_stop = 1

    for epoch in range(CFG.pre_train_epochs):
        print(f"Pre-train Epoch: {epoch + 1}")
        image_encoder.train()
        pre_train_epoch(image_encoder, train_loader, optimizer, lr_scheduler, step, epoch)

        image_encoder.eval()
        with torch.no_grad():
            valid_metrics = pre_valid_epoch(image_encoder, valid_loader, epoch)
        
        lr_scheduler.step(valid_metrics.avg_loss)

        if valid_metrics.F1 > best_F1:
          best_F1 = valid_metrics.F1
          consecutive = 0
          torch.save(image_encoder.state_dict(), "pre_train_200.pt")
          print("Saved Best Model!")

        elif consecutive < early_stop - 1:
          consecutive += 1

        else:
          break
  

In [12]:
image = None
while(image is None):
    image = cv2.imread(f"{CFG.image_path}/ImageCLEFmedCaption_2022_train_053449.jpg") # FIX colab timeout cv2 error
print(image.shape)
main()

(128, 128, 3)


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-rsb-weights/convnext_nano_d1h-7eb4bdea.pth" to /root/.cache/torch/hub/checkpoints/convnext_nano_d1h-7eb4bdea.pth


Pre-train Epoch: 1


  0%|          | 0/1172 [00:00<?, ?it/s]

  0%|          | 0/131 [00:00<?, ?it/s]

Saved Best Model!
Pre-train Epoch: 2


  0%|          | 0/1172 [00:00<?, ?it/s]

  0%|          | 0/131 [00:00<?, ?it/s]